<a href="https://colab.research.google.com/github/prashanth-ds-ml/Marketing_chat_bot/blob/main/Marketeer_Patched_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Marketeer — Conversational Marketing Bot (Patched)

**This version fixes video scripting returning empty content** by:
- Using Gemma-friendly prompting (no `system` role for JSON blocks)
- Strong JSON extraction + graceful fallback per beat
- REPL `/video` is fully wired to `make_video()`
- Includes a quick self‑test cell


In [ ]:
import sys, subprocess, platform
print(f'▶ Python: {sys.version.split()[0]}')
print(f'▶ Platform: {platform.platform()}')
print('\n▶ nvidia-smi:')
try:
    print(subprocess.check_output(['nvidia-smi'], text=True))
except Exception:
    print('(no GPU visible)')


▶ Python: 3.12.12
▶ Platform: Linux-6.6.105+-x86_64-with-glibc2.35

▶ nvidia-smi:
Sun Nov  2 10:35:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   31C    P0             53W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |               

## Install base libs
(If already installed, this is a no-op.)

In [ ]:
%pip -q install --upgrade pip
%pip -q install transformers accelerate sentence-transformers faiss-cpu pypdf textstat regex tiktoken rich


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.5 MB/s eta 0:00:00


In [ ]:
import torch, transformers, textwrap, re, json
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Any
from collections import deque
print({'torch': torch.__version__, 'transformers': transformers.__version__})


{'torch': '2.8.0+cu126', 'transformers': '4.57.1'}


In [ ]:
from typing import Dict, List
import re

PLATFORM_RULES: Dict[str, Dict[str, int]] = {
    'Instagram':   {'cap': 2200, 'hashtags_max': 5, 'emoji_max': 5},
    'Facebook':    {'cap': 125,  'hashtags_max': 0, 'emoji_max': 1},
    'LinkedIn':    {'cap': 3000, 'hashtags_max': 3, 'emoji_max': 2},
    'Google Ads':  {'cap': 90,   'hashtags_max': 0, 'emoji_max': 0},
    'Twitter/X':   {'cap': 280,  'hashtags_max': 2, 'emoji_max': 2},
}

BANNED_MAP = {
    r'\bguarantee(d|s)?\b': 'aim to',
    r'\bno[-\s]?risk\b': 'low risk',
    r'\bno[-\s]?questions[-\s]?asked\b': 'hassle-free',
    r'\b#?1\b': 'top-rated',
    r'\bbest\b': 'trusted',
    r'\bfastest\b': 'fast',
}
EMOJI_RX   = re.compile(r'[\U0001F300-\U0001FAFF\U00002700-\U000027BF]')
HASHTAG_RX = re.compile(r'(#\w+)')
SPACE_RX   = re.compile(r'\s+')

def _replace_banned(text: str, audit: List[str]) -> str:
    new = text
    for pat, repl in BANNED_MAP.items():
        if re.search(pat, new, flags=re.I):
            new = re.sub(pat, repl, new, flags=re.I)
            audit.append(f"Replaced banned phrasing '{pat}' -> '{repl}'.")
    return new

def _limit_hashtags(s: str, max_tags: int, audit: List[str]) -> str:
    tags = HASHTAG_RX.findall(s)
    if max_tags == 0 and tags:
        s2 = HASHTAG_RX.sub('', s)
        s2 = SPACE_RX.sub(' ', s2).strip()
        audit.append('Removed all hashtags per platform rules.')
        return s2
    if len(tags) <= max_tags:
        return s
    count = 0
    toks = s.split()
    for i, tok in enumerate(toks):
        if tok.startswith('#'):
            count += 1
            if count > max_tags:
                toks[i] = ''
    s2 = ' '.join(t for t in toks if t)
    audit.append(f'Trimmed hashtags to <= {max_tags}.')
    return s2

def _limit_emojis(s: str, max_emojis: int, audit: List[str]) -> str:
    if max_emojis < 0:
        return s
    emojis = EMOJI_RX.findall(s)
    if len(emojis) <= max_emojis:
        return s
    kept = 0; out = []
    for ch in s:
        if EMOJI_RX.match(ch):
            if kept < max_emojis:
                out.append(ch); kept += 1
        else:
            out.append(ch)
    audit.append(f'Trimmed emojis to <= {max_emojis}.')
    return ''.join(out)

def _ensure_keywords(s: str, keywords: List[str], cap: int, audit: List[str]) -> str:
    text = s
    for kw in (keywords or []):
        if kw.strip() and re.search(re.escape(kw), text, flags=re.I) is None:
            cand = (text + ' ' + kw).strip()
            if len(cand) <= cap:
                text = cand
            else:
                parts = text.split()
                if parts:
                    parts[-1] = kw
                    text = ' '.join(parts)
            audit.append(f'Inserted missing keyword: {kw}')
    return text

def _pick_cta(cta_strength: str) -> str:
    bank = {
        'soft':   ['Learn more','See how it works','Try it free'],
        'medium': ['Get started today','Start now'],
        'hard':   ['Start your free trial now','Buy now','Sign up now'],
    }.get(cta_strength, ['Learn more'])
    return bank[0]

def _ensure_cta(text: str, cta_strength: str, audit: List[str]) -> str:
    if re.search(r'\b(learn more|start|try|buy|sign up|get started|discover|explore)\b', text, flags=re.I):
        return text
    cta = _pick_cta(cta_strength)
    audit.append(f"Added CTA: '{cta}'.")
    return (text + (' ' if not text.endswith(('.', '!', '?')) else ' ') + cta).strip()

def _smart_trim(text: str, cap: int, preserve_tail: str = '') -> str:
    if len(text) <= cap:
        return text
    reserve = len(preserve_tail) + (1 if preserve_tail and not text.endswith(' ') else 0)
    hard = max(0, cap - reserve)
    trimmed = text[:hard].rstrip()
    if preserve_tail:
        if not trimmed.endswith(('.', '!', '?')):
            trimmed = trimmed.rstrip(',;:-')
        trimmed = (trimmed + ' ' + preserve_tail).strip()
    return trimmed[:cap]

def apply_validators(text: str, platform: str, cap: int, cta_strength: str, keywords: List[str]):
    audit: List[str] = []
    s = text.strip()
    s = _replace_banned(s, audit)
    s = _ensure_cta(s, cta_strength, audit)
    s = _ensure_keywords(s, keywords, 10**9, audit)
    tail_cta = ''
    m = re.search(r'(learn more|start your free trial(?: now)?|start now|try it free|get started(?: today)?|buy now|sign up(?: now)?)$', s, flags=re.I)
    if m:
        tail_cta = m.group(0)
    s = _smart_trim(s, cap, preserve_tail=tail_cta)
    s = _ensure_keywords(s, keywords, cap, audit)
    rule = PLATFORM_RULES.get(platform, PLATFORM_RULES['Instagram'])
    s = _limit_hashtags(s, rule['hashtags_max'], audit)
    s = _limit_emojis(s, rule['emoji_max'], audit)
    if len(s) > cap:
        s = s[:cap].rstrip()
        audit.append(f'Force-clipped to {cap} chars.')
    return s, audit


In [ ]:
class ShortWindowMemory:
    def __init__(self, k: int = 3):
        self.window = deque(maxlen=k)
    def add(self, user: str, assistant: str):
        self.window.append({'user': user, 'assistant': assistant})
    def text(self) -> str:
        if not self.window:
            return ''
        lines = []
        for t in self.window:
            lines.append(f"Human: {t['user']}")
            lines.append(f"AI: {t['assistant']}")
        return '\n'.join(lines)
    def reset(self):
        self.window.clear()


In [ ]:
SESSION_PROFILE: Dict[str, Any] = {
    'brand': '', 'product': '', 'audience': '', 'voice': '', 'facts': {}
}
def remember(k: str, v: str):
    if k in ('brand','product','audience','voice'):
        SESSION_PROFILE[k] = v
    else:
        SESSION_PROFILE['facts'][k] = v
    return SESSION_PROFILE
def forget(k: str):
    if k in ('brand','product','audience','voice'):
        SESSION_PROFILE[k] = ''
    else:
        SESSION_PROFILE['facts'].pop(k, None)
    return SESSION_PROFILE
def facts_dump() -> str:
    f = [f"- brand: {SESSION_PROFILE.get('brand','')}",
         f"- product: {SESSION_PROFILE.get('product','')}",
         f"- audience: {SESSION_PROFILE.get('audience','')}",
         f"- voice: {SESSION_PROFILE.get('voice','')}"]
    if SESSION_PROFILE['facts']:
        f.append('- facts:')
        for k,v in SESSION_PROFILE['facts'].items():
            f.append(f"  • {k}: {v}")
    return '\n'.join(f)
BASE_GUIDANCE = (
    'You are Marketeer, a concise, benefit-first marketing copywriter. '
    'Respect the platform\'s character cap, include required keywords, and end with a clear but compliant CTA. '
    "Avoid absolute claims like 'guaranteed', '#1', or 'best'. Prefer modest, evidence-backed phrasing."
)
def _profile_block() -> str:
    lines = []
    if any(SESSION_PROFILE.values()):
        lines.append('Session profile:')
        if SESSION_PROFILE.get('brand'):   lines.append(f"- Brand: {SESSION_PROFILE['brand']}")
        if SESSION_PROFILE.get('product'): lines.append(f"- Product: {SESSION_PROFILE['product']}")
        if SESSION_PROFILE.get('audience'):lines.append(f"- Audience: {SESSION_PROFILE['audience']}")
        if SESSION_PROFILE.get('voice'):   lines.append(f"- Voice: {SESSION_PROFILE['voice']}")
        if SESSION_PROFILE['facts']:
            lines.append('- Key facts:')
            for k,v in SESSION_PROFILE['facts'].items():
                lines.append(f"  • {k}: {v}")
    return '\n'.join(lines) if lines else '[no session profile]'
import textwrap
def build_prompt(user_input: str, platform: str, tone: str, cta_strength: str, cap: int,
                 keywords: List[str], history_text: str = '') -> str:
    kw = ', '.join(keywords) if keywords else '(none)'
    profile = _profile_block()
    return textwrap.dedent(f"""
    {BASE_GUIDANCE}

    Context (recent conversation, if any):
    {history_text if history_text else '[no prior turns in memory]'}

    {profile}

    Task:
    - Platform: {platform}
    - Tone: {tone}
    - CTA strength: {cta_strength}
    - Character cap: {cap}
    - Required keywords: {kw}

    User request:
    {user_input}

    Instructions:
    - Be benefit-first and platform-appropriate.
    - Keep within the character cap (hard limit {cap} chars).
    - Include all required keywords (if any).
    - Close with a clear CTA matching CTA strength.
    - Avoid banned claims ('guaranteed', '#1', 'best').

    Return only the marketing copy (no preamble).
    """).strip()


In [ ]:
import getpass
MODEL_ID = 'google/gemma-2-2b-it'
DTYPE = 'bfloat16'
try:
    token = getpass.getpass('Enter HF token (press Enter to skip): ')
    HF_TOKEN = token.strip() or None
except Exception:
    HF_TOKEN = None
torch_dtype = {'bfloat16': torch.bfloat16, 'float16': torch.float16}.get(DTYPE, torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, dtype=torch_dtype, device_map='auto', token=HF_TOKEN)
print({'model': MODEL_ID, 'dtype': str(torch_dtype).replace('torch.', '')})


Enter HF token (press Enter to skip): ··········


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

{'model': 'google/gemma-2-2b-it', 'dtype': 'bfloat16'}


In [ ]:
memory = ShortWindowMemory(k=3)
DEFAULTS = {'platform': 'Instagram', 'tone': 'friendly, energetic', 'cta': 'soft'}
def _generate(prompt_text: str, max_new_tokens=180, temperature=0.7, top_p=0.9, repetition_penalty=1.1):
    messages = [{'role': 'user', 'content': prompt_text}]
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors='pt').to(model.device)
    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    gen_ids = out[0][input_ids.shape[-1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
def send(user: str, platform: str=None, tone: str=None, cta_strength: str=None, cap: int=None, keywords: List[str]=None,
         max_new_tokens=180, temperature=0.7, top_p=0.9, repetition_penalty=1.1):
    platform = platform or DEFAULTS['platform']
    tone = tone or DEFAULTS['tone']
    cta_strength = cta_strength or DEFAULTS['cta']
    cap = cap or PLATFORM_RULES.get(platform, PLATFORM_RULES['Instagram'])['cap']
    keywords = keywords or []
    prompt_text = build_prompt(user, platform, tone, cta_strength, cap, keywords, memory.text())
    raw = _generate(prompt_text, max_new_tokens, temperature, top_p, repetition_penalty)
    final, audit = apply_validators(raw, platform, cap, cta_strength, keywords)
    memory.add(user, final)
    return {'raw': raw, 'final': final, 'audit': audit, 'cap': cap}


In [ ]:
VIDEO_BLUEPRINTS = {
    'short_ad': ['Hook (2-3s)','Problem (3-5s)','Product intro (4-6s)','Key benefit (4-6s)','CTA (2-3s)'],
    'ugc_review': ['Relatable hook','Pain point','Discovery','Feature demo','Social proof','CTA'],
    'how_to': ['Teaser result','Step 1','Step 2','Step 3','Recap + CTA'],
}
def plan_video(blueprint: str='short_ad', duration_sec: int=20, product_brief: str='') -> Dict[str, Any]:
    if blueprint not in VIDEO_BLUEPRINTS:
        raise ValueError(f"Unknown blueprint '{blueprint}'. Options: {list(VIDEO_BLUEPRINTS.keys())}")
    beats = VIDEO_BLUEPRINTS[blueprint]
    per_beat = max(2, duration_sec // max(1, len(beats)))
    plan = []
    for i, beat in enumerate(beats, 1):
        plan.append({
            'order': i,
            'beat': beat,
            'time_window': f"{(i-1)*per_beat:02d}-{min(i*per_beat, duration_sec):02d}s",
        })
    return {
        'blueprint': blueprint,
        'duration_sec': duration_sec,
        'beats': plan,
        'product_brief': product_brief or '(use chat context)',
    }


In [ ]:
import json as _json, re as _re

def _session_context_text():
    history_text = memory.text()
    lines = []
    lines.append('=== SESSION PROFILE ===')
    if SESSION_PROFILE.get('brand'):   lines.append(f"Brand: {SESSION_PROFILE['brand']}")
    if SESSION_PROFILE.get('product'): lines.append(f"Product: {SESSION_PROFILE['product']}")
    if SESSION_PROFILE.get('audience'):lines.append(f"Audience: {SESSION_PROFILE['audience']}")
    if SESSION_PROFILE.get('voice'):   lines.append(f"Preferred Voice: {SESSION_PROFILE['voice']}")
    if SESSION_PROFILE.get('facts'):
        lines.append('Facts:')
        for k,v in SESSION_PROFILE['facts'].items():
            lines.append(f"- {k}: {v}")
    lines.append('\n=== RECENT CONVERSATION ===')
    lines.append(history_text if history_text else '[none]')
    return '\n'.join(lines)

def _decode_tail(out_ids, start_idx):
    return tokenizer.decode(out_ids[0][start_idx:], skip_special_tokens=True).strip()

def _just_user_prompt(prompt_text: str, max_new_tokens=320):
    messages = [{'role':'user','content': prompt_text}]
    input_ids = tokenizer.apply_chat_template(
        messages, tokenize=True, add_generation_prompt=True, return_tensors='pt'
    ).to(model.device)
    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            max_new_tokens=max_new_tokens,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    return _decode_tail(out, input_ids.shape[-1])

def _extract_json(gen_text: str):
    try:
        return _json.loads(gen_text)
    except Exception:
        pass
    m = _re.search(r'\{.*\}', gen_text, flags=_re.S)
    if m:
        try:
            return _json.loads(m.group(0))
        except Exception:
            pass
    return None

def _fallback_block(beat_title: str):
    short = beat_title.split('(')[0].strip()
    return {
        'voiceover': f"{short}: naturally delicious, try it today.",
        'on_screen': (short[:32] or 'Fresh & Natural'),
        'shots': ['Close-up product', 'Serving scoop', 'Happy bite'],
        'broll': ['Farm/ingredient cutaways', 'Pouring/serving'],
        'captions': ['Naturally made ice cream', 'From local farms'],
    }

def script_video_from_plan(plan: dict, style: str = 'friendly, energetic', platform: str = 'Instagram', debug_first=False):
    context = _session_context_text()
    blueprint = plan.get('blueprint', '?')
    duration  = plan.get('duration_sec', 20)
    beats     = plan.get('beats', [])
    scripted_beats = []

    for idx, b in enumerate(beats):
        brief = (
            context + '\n\n'
            '=== VIDEO BLUEPRINT ===\n'
            f'Type: {blueprint} | Duration: {duration}s\n'
            f"Current beat: {b['order']} — {b['beat']} ({b['time_window']})\n\n"
            'Write concise items with the following constraints:\n'
            '- voiceover: <= 18 words, benefits-first, natural.\n'
            '- on_screen: <= 36 characters, punchy overlay text.\n'
            '- shots: 3 ideas, short imperatives (e.g., "Close-up pour").\n'
            '- broll: 2 ideas, short.\n'
            '- captions: 1–2 lines, each <= 40 characters.\n\n'
            f'Platform: {platform}\n'
            f'Style/Tone: {style}\n\n'
            'Return ONLY valid JSON with keys:\n'
            '{\n  "voiceover": "string",\n  "on_screen": "string",\n  "shots": ["...", "...", "..."],\n  "broll": ["...", "..."],\n  "captions": ["...", "..."]\n}'
        )
        gen_text = _just_user_prompt(brief, max_new_tokens=320)
        if debug_first and idx == 0:
            print('RAW (beat 1):\n', gen_text)
        data = _extract_json(gen_text)
        if not data:
            data = _fallback_block(b.get('beat','Beat'))

        scripted_beats.append({
            'order': b.get('order'),
            'time_window': b.get('time_window'),
            'beat': b.get('beat'),
            'voiceover': data.get('voiceover','') or _fallback_block(b.get('beat',''))['voiceover'],
            'on_screen': data.get('on_screen','') or _fallback_block(b.get('beat',''))['on_screen'],
            'shots': (data.get('shots') or _fallback_block(b.get('beat',''))['shots'])[:3],
            'broll': (data.get('broll') or _fallback_block(b.get('beat',''))['broll'])[:2],
            'captions': (data.get('captions') or _fallback_block(b.get('beat',''))['captions'])[:2],
        })

    return {
        'blueprint': blueprint,
        'duration_sec': duration,
        'style': style,
        'platform': platform,
        'product_brief': plan.get('product_brief',''),
        'script': scripted_beats
    }

def make_video(plan_or_blueprint='short_ad', duration=20, product_brief='', style='friendly, energetic', platform='Instagram', debug_first=False):
    if isinstance(plan_or_blueprint, dict):
        plan = plan_or_blueprint
    else:
        plan = plan_video(plan_or_blueprint, duration, product_brief)
    return script_video_from_plan(plan, style=style, platform=platform, debug_first=debug_first)


In [ ]:
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.markdown import Markdown
import re, json as _j

console = Console()
def _print_header():
    t = Table(title='Marketeer — REPL (Patched)', show_lines=False)
    t.add_column('Setting', style='cyan', no_wrap=True)
    t.add_column('Value', style='white')
    t.add_row('Model', MODEL_ID)
    t.add_row('Platform', 'Instagram')
    t.add_row('Tone', 'friendly, energetic')
    t.add_row('CTA', 'soft')
    console.print(t)
    console.print(Markdown(
        '**Commands**\n'
        '- `/remember key=value`, `/forget key`, `/facts`\n'
        '- `/video blueprint=short_ad duration=20 style=warm platform=Instagram`\n'
        '- Type any prompt to generate copy.'
    ))

def _parse_kv(line: str):
    parts = re.findall(r'(\w+)=(".*?"|\'.*?\'|\S+)', line)
    out = {}
    for k, v in parts:
        v = v.strip().strip('"').strip("'")
        out[k] = v
    return out

def repl():
    _print_header()
    while True:
        try:
            line = console.input('[bold magenta]You[/bold magenta]: ').strip()
        except (KeyboardInterrupt, EOFError):
            console.print('\n[yellow]Bye.[/yellow]'); break
        if not line:
            continue
        low = line.lower()
        if low in ('/exit','exit','quit','/quit'):
            console.print('[yellow]Bye.[/yellow]'); break
        if low.startswith('/facts'):
            console.print(Panel(facts_dump() or '(no facts)', title='Session Facts')); continue
        if low.startswith('/forget'):
            kv = _parse_kv(line)
            for k in kv.keys(): forget(k)
            console.print(Panel('Updated facts.', title='OK')); continue
        if low.startswith('/remember'):
            kv = _parse_kv(line)
            if not kv and '=' in line:
                raw = line.split(None, 1)[1]
                k,v = raw.split('=',1)
                remember(k.strip(), v.strip())
            else:
                for k,v in kv.items(): remember(k, v)
            console.print(Panel('Saved.', title='OK')); continue
        if low.startswith('/video'):
            kv = _parse_kv(line)
            blueprint = kv.get('blueprint','short_ad')
            duration  = int(kv.get('duration','20'))
            style     = kv.get('style','friendly, energetic')
            platformV = kv.get('platform','Instagram')
            brief     = SESSION_PROFILE.get('product','') or SESSION_PROFILE.get('brand','') or 'marketing video'
            plan = plan_video(blueprint, duration, brief)
            script = make_video(plan, style=style, platform=platformV, debug_first=True)
            console.print(Panel(_j.dumps(script, indent=2), title='Video Script'))
            continue
        # normal prompt
        result = send(line)
        console.print(Panel(result['final'], title='Response', subtitle=f"len={len(result['final'])}/{result['cap']}"))
        if result['audit']:
            md = '\n'.join([f"- {a}" for a in result['audit']])
            console.print(Panel(md, title='Audit trail'))
        else:
            console.print('[dim]No edits needed.[/dim]')

repl()


    Marketeer — REPL (Patched)     
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Setting  ┃ Value                ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ Model    │ google/gemma-2-2b-it │
│ Platform │ Instagram            │
│ Tone     │ friendly, energetic  │
│ CTA      │ soft                 │
└──────────┴──────────────────────┘

Commands                                                                                                           

 • /remember key=value, /forget key, /facts                                                                        
 • /video blueprint=short_ad duration=20 style=warm platform=Instagram                                             
 • Type any prompt to generate copy.

You:

hi buddy


╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ Hey there! 👋  What's on your mind today? 😊                                                                    │
│                                                                                                                 │
│                                                                                                                 │
│ Let's connect and make something awesome happen. 🚀  I specialize in crafting engaging social media content     │
│ that drives real results.  Whether you need eye-catching captions, killer visuals, or strategic planning, I can │
│ help you level up your brand. 💪 Let's chat about how we can boost your online presence together! 😉 Learn more │
╰───────────────────────────────────────────────── len=374/2200 ──────────────────────────────────────────────────╯

...

You:

ok now i am launching a Indian namkeen brand called little habbit, lets make a brand taglie and blogs


╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ Craving a taste of authentic Indian snacking? 😋 Little Habit brings you artisanal namkeens, handcrafted with   │
│ love using traditional recipes and premium ingredients.                                                         │
│                                                                                                                 │
│ Imagine crunchy bites bursting with flavour - that's Little Habit! Each batch is lovingly prepared to deliver   │
│ the perfect balance of crunch and spice.  🤤                                                                    │
│                                                                                                                 │
│ Explore our range of irresistible snacks and discover your new favourite treat.  👉  Visit our website today!   │
╰───────────────────────────────────────────────── len=430/2200 ──────────────────────────────────────────────────╯

No edits needed.

You:

## Quick self‑test (optional)

In [ ]:
# Uncomment to smoke test video scripting
# remember('brand','FrostFields')
# remember('product','Natural fruit ice cream')
# remember('origin','Local farms')
# plan = plan_video('short_ad', 20, 'Natural ice cream with coconut & apple')
# vid = make_video(plan, style='warm, wholesome', platform='Instagram', debug_first=True)
# import json as j; print(j.dumps(vid, indent=2))
